In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

# Formateo de decimales
pd.options.display.float_format = '{:,.3f}'.format

print("pd.__version__", pd.__version__)
print("np.__version__", np.__version__)

pd.__version__ 1.2.4
np.__version__ 1.20.2


In [5]:
df_tips = sns.load_dataset('tips')
print(df_tips.shape)
df_tips.head()

(244, 7)


,total_bill,tip,sex,smoker,day,time,size
0,16.990,1.010,Female,No,Sun,Dinner,2
1,10.340,1.660,Male,No,Sun,Dinner,3
2,21.010,3.500,Male,No,Sun,Dinner,3
3,23.680,3.310,Male,No,Sun,Dinner,2
4,24.590,3.610,Female,No,Sun,Dinner,4


In [6]:
df_tips.describe(include='all')

,total_bill,tip,sex,smoker,day,time,size
count,244.000,244.000,244,244,244,244,244.000
unique,NaN,NaN,2,2,4,2,NaN
top,NaN,NaN,Male,No,Sat,Dinner,NaN
freq,NaN,NaN,157,151,87,176,NaN
mean,19.786,2.998,NaN,NaN,NaN,NaN,2.570
std,8.902,1.384,NaN,NaN,NaN,NaN,0.951
min,3.070,1.000,NaN,NaN,NaN,NaN,1.000
25%,13.348,2.000,NaN,NaN,NaN,NaN,2.000
50%,17.795,2.900,NaN,NaN,NaN,NaN,2.000
75%,24.127,3.562,NaN,NaN,NaN,NaN,3.000


In [14]:
# Sirve para identificar rapidamente valores categoricos
df_tips.nunique()

total_bill    229
tip           123
sex             2
smoker          2
day             4
time            2
size            6
dtype: int64

In [18]:
# Conteo de datos en una columna
df_tips['day'].value_counts().to_frame()

,day
Sat,87
Sun,76
Thur,62
Fri,19


In [16]:
# Agrupacion por columna y aplicando la media
df_tips.groupby('sex').mean()

,total_bill,tip,size
sex,,,
Male,20.744,3.090,2.631
Female,18.057,2.833,2.460


In [19]:
# Creando una nueva columna con operaciones entre columna
df_tips['prct_tip'] = df_tips['tip'] / df_tips['total_bill']
df_tips.tail()

,total_bill,tip,sex,smoker,day,time,size,prct_tip
239,29.030,5.920,Male,No,Sat,Dinner,3,0.204
240,27.180,2.000,Female,Yes,Sat,Dinner,2,0.074
241,22.670,2.000,Male,Yes,Sat,Dinner,2,0.088
242,17.820,1.750,Male,No,Sat,Dinner,2,0.098
243,18.780,3.000,Female,No,Thur,Dinner,2,0.160


In [20]:
# Agrupacion por columna, pivoteando otras columnas y aplicando funciones
df_tips.groupby('sex')[['total_bill', 'prct_tip']].describe()

total_bill                                                prct_tip  \
            count   mean   std   min    25%    50%    75%    max    count   
sex                                                                         
Male      157.000 20.744 9.246 7.250 14.000 18.350 24.710 50.810  157.000   
Female     87.000 18.057 8.009 3.070 12.750 16.400 21.520 44.300   87.000   

                                                  
        mean   std   min   25%   50%   75%   max  
sex                                               
Male   0.158 0.065 0.036 0.121 0.153 0.186 0.710  
Female 0.166 0.054 0.056 0.140 0.156 0.194 0.417

In [21]:
# Aplicando funciones a columnas
def mean_eur2usd(x):
    y = np.mean(x) * 1.12
    return y


# Agrupacion por una columna, pivoteando otras columnas aplicando funciones
print(df_tips.groupby('sex')[['total_bill', 'prct_tip']].apply(mean_eur2usd))
# Agrupacion por mas de una columna, pivoteando otras columnas aplicando funciones
print(df_tips.groupby(['sex', 'time'])[['total_bill', 'prct_tip']].apply(mean_eur2usd))
# Agrupacion por mas de una columna, aplicando una lambda
print(df_tips.groupby(['sex', 'time'])[['total_bill', 'prct_tip']].apply(lambda x: np.mean(x) * 1.12))
# Agrupacion por mas de una columna, aplicando mas de una funcion
print(df_tips.groupby(['sex', 'time'])[['total_bill', 'prct_tip']].aggregate([np.mean, np.max, np.sum]))

        total_bill  prct_tip
sex                         
Male        23.233     0.177
Female      20.224     0.186
               total_bill  prct_tip
sex    time                        
Male   Lunch       20.214     0.186
       Dinner      24.037     0.174
Female Lunch       18.300     0.182
       Dinner      21.519     0.190
               total_bill  prct_tip
sex    time                        
Male   Lunch       20.214     0.186
       Dinner      24.037     0.174
Female Lunch       18.300     0.182
       Dinner      21.519     0.190
              total_bill                  prct_tip             
                    mean   amax       sum     mean  amax    sum
sex    time                                                    
Male   Lunch      18.048 41.190   595.600    0.166 0.266  5.481
       Dinner     21.461 50.810 2,661.220    0.155 0.710 19.270
Female Lunch      16.339 43.110   571.870    0.162 0.259  5.680
       Dinner     19.213 44.300   999.080    0.169 0.417  8.805


In [22]:
# Usando un diccionario para definir todas las funciones a aplicar a la agrupacion
dict_agg = {
    'tip': [min, max],
    'total_bill': [np.mean, mean_eur2usd],
    'prct_tip': [lambda x: np.mean(x) * 1.12, np.sum]
}

df_tips.groupby(['sex', 'time'])[['tip', 'total_bill', 'prct_tip', ]].aggregate(dict_agg)


tip        total_bill                prct_tip       
                min    max       mean mean_eur2usd <lambda_0>    sum
sex    time                                                         
Male   Lunch  1.440  6.700     18.048       20.214      0.186  5.481
       Dinner 1.000 10.000     21.461       24.037      0.174 19.270
Female Lunch  1.250  5.170     16.339       18.300      0.182  5.680
       Dinner 1.000  6.500     19.213       21.519      0.190  8.805

In [24]:
# Filtrando agrupacion por una funcion
def f_filter(x):
    return mean_eur2usd(x['total_bill'].mean()) > 20


df_filtered = df_tips.groupby(['sex', 'time']).filter(f_filter)
print(df_filtered.shape)
df_filtered.head()

(209, 8)


,total_bill,tip,sex,smoker,day,time,size,prct_tip
0,16.990,1.010,Female,No,Sun,Dinner,2,0.059
1,10.340,1.660,Male,No,Sun,Dinner,3,0.161
2,21.010,3.500,Male,No,Sun,Dinner,3,0.167
3,23.680,3.310,Male,No,Sun,Dinner,2,0.140
4,24.590,3.610,Female,No,Sun,Dinner,4,0.147


In [25]:
# Contando registros a partir de la agrupacion
df_filtered.groupby(['sex', 'time']).count()


total_bill  tip  smoker  day  size  prct_tip
sex    time                                                
Male   Lunch           33   33      33   33    33        33
       Dinner         124  124     124  124   124       124
Female Lunch            0    0       0    0     0         0
       Dinner          52   52      52   52    52        52

In [28]:
# Creando una columna con el valor 1, que servira para hacer conteos en las agrupaciones
df_tips['ones'] = 1
print(df_tips.head())
# Usando la columna de ones para sumar en la agrupacion, suponiendo que 1 registro equivale a un dato valido
df_g = df_tips.groupby(['sex', 'smoker'])[['ones']].sum()
df_g

   total_bill   tip     sex smoker  day    time  size  prct_tip  ones
0      16.990 1.010  Female     No  Sun  Dinner     2     0.059     1
1      10.340 1.660    Male     No  Sun  Dinner     3     0.161     1
2      21.010 3.500    Male     No  Sun  Dinner     3     0.167     1
3      23.680 3.310    Male     No  Sun  Dinner     2     0.140     1
4      24.590 3.610  Female     No  Sun  Dinner     4     0.147     1


ones
sex    smoker      
Male   Yes       60
       No        97
Female Yes       33
       No        54

In [29]:
# Usando la agrupacion formada, y aplicando sobre alguno de los niveles de la agurpacion una funcion
# Para este caso se esta sacando el porcentaje de participacion
df_g.groupby(level=0).apply(lambda x: x.sum() * 100)



,ones
sex,
Male,15700
Female,8700


In [34]:
# pd.cut // Permite hacer una segmentacion de datos a apartir de los parametros
# Conteo de los valores que pertenecen dentro de cada rango
print(pd.cut(df_tips['total_bill'], bins=[3, 18, 35, 60]).value_counts())

(3, 18]     127
(18, 35]    101
(35, 60]     16
Name: total_bill, dtype: int64


In [35]:
# Segmentacion de datos , agrupando en rangos un valor numerico haciendolo categorico
# bins = [rangos a cortar la agrupacion]
# Agrega una columna con las egmentaciones
df_tips['bin_total'] = pd.cut(df_tips['total_bill'], bins=[3, 18, 35, 60])
df_tips.head()

,total_bill,tip,sex,smoker,day,time,size,prct_tip,ones,bin_total
0,16.990,1.010,Female,No,Sun,Dinner,2,0.059,1,"(3, 18]"
1,10.340,1.660,Male,No,Sun,Dinner,3,0.161,1,"(3, 18]"
2,21.010,3.500,Male,No,Sun,Dinner,3,0.167,1,"(18, 35]"
3,23.680,3.310,Male,No,Sun,Dinner,2,0.140,1,"(18, 35]"
4,24.590,3.610,Female,No,Sun,Dinner,4,0.147,1,"(18, 35]"


In [37]:
# Agrupando por mas de una columna, usando la columna ones para el conteo
df_tips.groupby(['time', 'bin_total'])[['ones']].count()

ones
time   bin_total      
Lunch  (3, 18]      45
       (18, 35]     21
       (35, 60]      2
Dinner (3, 18]      82
       (18, 35]     80
       (35, 60]     14

In [40]:
# Misma operacion de agrupacion, pero obteniendo la participacion por el nivel 0 de agrupacion
df_partipacion = df_tips.groupby(['time', 'bin_total'])[['ones']].count().groupby(level=0).apply(
    lambda x:
    x / x.sum() * 100
)
df_partipacion

'\nTABLAS PIVOTE\n'

In [42]:
"""
TABLAS PIVOTE
"""
# Agrupacion por columnas, y aplicando funciones a partir de los valores de otra columna
df_tips.groupby(['sex', 'time'])[['total_bill']].mean()

total_bill
sex    time              
Male   Lunch       18.048
       Dinner      21.461
Female Lunch       16.339
       Dinner      19.213

In [44]:
# Añadinedo indices numericos al dataframe, de forma que confireve la agrupacion a una tabla
df_gp = df_tips.groupby(['sex', 'time'])[['total_bill']].mean().reset_index()
df_gp

,sex,time,total_bill
0,Male,Lunch,18.048
1,Male,Dinner,21.461
2,Female,Lunch,16.339
3,Female,Dinner,19.213


In [45]:
# pivot_table(values=['valores a operar'], index=[filas o nivel de agrupacion], columns=[columnas a pivotear], aggfunc=[funciones a operar])
# Si no se especifica el valor de aggfunc realizara la media por default
df_gp.pivot_table(values='total_bill', index='sex', columns='time')

time,Lunch,Dinner
sex,,
Male,18.048,21.461
Female,16.339,19.213


In [46]:
# pivot_table(values=['valores a operar'], index=[filas o nivel de agrupacion], columns=[columnas a pivotear], aggfunc=[funciones a operar])
# Si no se especifica el valor de aggfunc realizara la media por default
df_tips.pivot_table(values='total_bill', index='sex', columns='time')

time,Lunch,Dinner
sex,,
Male,18.048,21.461
Female,16.339,19.213


In [56]:
# Tabla pivote o dinamica
df_pivot = df_tips.pivot_table(values=['total_bill', 'tip'],
                               index=['sex', 'smoker'],
                               columns=['time'],
                               aggfunc=[np.median, np.std])
df_pivot
#df_tips

median                            std                         
                 tip        total_bill          tip        total_bill       
time           Lunch Dinner      Lunch Dinner Lunch Dinner      Lunch Dinner
sex    smoker                                                               
Male   Yes     2.200  3.000     16.000 23.100 1.096  1.596      6.852 10.250
       No      2.405  3.000     16.975 18.290 1.486  1.497      8.739  8.748
Female Yes     2.500  3.000     14.845 16.320 1.057  1.306      9.426  9.290
       No      2.000  3.000     13.420 17.310 1.078  1.118      6.759  7.298

In [53]:
# Convirtiendo la agrupacion a tabla
# .unstack() // traspone datos de columnas a filas
# .reset_index() // agrega nuevos indices numericos
df_pivot.unstack().reset_index()

,level_0,level_1,time,sex,0
0,median,tip,Lunch,Male,2.310
1,median,tip,Lunch,Female,2.010
2,median,tip,Dinner,Male,3.000
3,median,tip,Dinner,Female,3.000
4,median,total_bill,Lunch,Male,16.580
5,median,total_bill,Lunch,Female,13.420
6,median,total_bill,Dinner,Male,19.630
7,median,total_bill,Dinner,Female,17.190
8,std,tip,Lunch,Male,1.329
9,std,tip,Lunch,Female,1.075
